In [1]:
!nvidia-smi

Wed Feb  9 13:48:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms as T
from torchvision import models
import torchvision
 
from torch.utils.data import DataLoader,Dataset, random_split, WeightedRandomSampler
 
import cv2
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
cd gdrive/"My Drive"

/content/gdrive/My Drive


In [5]:
data_path = 'surfacique_no_ss_class'
!ls {data_path}

gpe_Agde_évolué			  gpe_Macédoine
gpe_Angleterre			  gpe_Mecklenburg_obliques
gpe_Auvernier			  gpe_Mecklenburg_triangles
gpe_barques_hongroises		  gpe_Mitterkirchen
gpe_B_Danube			  gpe_Mola
gpe_Bignan			  gpe_Moora
gpe_brassart_Ha			  gpe_morave
gpe_c_Danube			  gpe_Morges
gpe_chevrons			  gpe_nord
gpe_Clans			  gpe_Ph_0
gpe_Corcelettes			  gpe_Pologne
gpe_coubes_verticales		  gpe_Pologne_Ha
gpe_Dhrovice			  gpe_réniforme
gpe_Estavayer			  gpe_Rosan
gpe_geométrique_et_anthopomorphe  gpe_SAY
gpe_Haguenau			  gpe_Subingen
gpe_jambières_barques_et_lignes   gpe_tampons
gpe_jambières_triangles		  gpe_tores
gpe_kasidol			  gpe_triangles_Hongrie
gpe_Leupolz			  gpe_Ukraine
gpe_Locarno			  gpe_Vinol
gpe_LT_A			  gpe_Wabern
gpe_Lunebourg			  gpe_Zerba


In [6]:
normalize = T.Normalize(mean=[0.5], std=[0.5])
transform = T.Compose([
                        # T.Grayscale(num_output_channels=1),                             
                        T.RandomHorizontalFlip(),
                        T.RandomPerspective(distortion_scale=0.05, p=0.5),
                        T.ToTensor(),
                        normalize,
                      ])

In [7]:
image_folder_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=transform)

In [8]:
#Splitting dataset to train and validation
dataset_size = image_folder_dataset.__len__()
train_count = int(dataset_size * 0.7)
val_count = dataset_size - train_count
train_dataset, valid_dataset = random_split(image_folder_dataset, [train_count, val_count])
y_train_indices = train_dataset.indices
y_train = [image_folder_dataset.targets[i] for i in y_train_indices]
class_sample_count = np.array(
    [len(np.where(y_train == t)[0]) for t in np.unique(y_train)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_dataloader = DataLoader(train_dataset, batch_size=16, sampler=sampler, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, pin_memory=True, shuffle=True)

In [9]:
# valid_dataloader.

SyntaxError: ignored

In [10]:
net = models.resnet18(pretrained=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.cuda() if device else net

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 512*512)
net.fc = net.fc.cuda()

In [12]:
net.load_state_dict(torch.load('resnet.pt'))

<All keys matched successfully>

In [13]:
n_epochs = 14
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_dataloader)

for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_dataloader):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = net(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (valid_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(valid_dataloader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    net.train()

Epoch 1

Epoch [1/14], Step [0/217], Loss: 2.9994
Epoch [1/14], Step [20/217], Loss: 3.9711
Epoch [1/14], Step [40/217], Loss: 3.0793
Epoch [1/14], Step [60/217], Loss: 4.5379
Epoch [1/14], Step [80/217], Loss: 3.3986
Epoch [1/14], Step [100/217], Loss: 4.1929
Epoch [1/14], Step [120/217], Loss: 3.9690
Epoch [1/14], Step [140/217], Loss: 3.7702
Epoch [1/14], Step [160/217], Loss: 3.4609
Epoch [1/14], Step [180/217], Loss: 3.1508
Epoch [1/14], Step [200/217], Loss: 2.2006

train-loss: 3.7129, train-acc: 83.1841
validation loss: 3.8640, validation acc: 72.8437

Improvement-Detected, save-model
Epoch 2

Epoch [2/14], Step [0/217], Loss: 3.2345
Epoch [2/14], Step [20/217], Loss: 3.9111
Epoch [2/14], Step [40/217], Loss: 3.3963
Epoch [2/14], Step [60/217], Loss: 2.6140
Epoch [2/14], Step [80/217], Loss: 4.5213
Epoch [2/14], Step [100/217], Loss: 4.6571
Epoch [2/14], Step [120/217], Loss: 3.0880
Epoch [2/14], Step [140/217], Loss: 3.1186
Epoch [2/14], Step [160/217], Loss: 3.5111
Epoch [2/14

In [ ]:
# from torchsummary import summary
# summary(net,input_size=(3,512,512))

# from torch.utils.tensorboard import SummaryWriter
# writer=SummaryWriter('/content/logsdir')

In [ ]:
# !tensorboard --logdir content/logsdir

In [ ]:
# image_folder_dataset.class_to_idx.keys()

In [ ]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sn
# import pandas as pd

# y_pred = []
# y_true = []

# # iterate over test data
# with torch.no_grad():
#   net.eval()

#   for data_t, target_t in (valid_dataloader):
#           data_t, target_t = data_t.to(device), target_t.to(device)
#           outputs_t = net(data_t)
#           y_pred.extend(outputs_t) # Save Prediction
          
#           target_t = target_t.data.cpu().numpy()
#           y_true.extend(target_t) # Save Truth

#   # constant for classes
#   classes = image_folder_dataset.class_to_idx.keys()

#   # Build confusion matrix
#   cf_matrix = confusion_matrix(y_true, y_pred)
#   df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
#                       columns = [i for i in classes])
#   plt.figure(figsize = (12,7))
#   sn.heatmap(df_cm, annot=True)
#   plt.savefig('output.png')